In [36]:
import numpy as np
import pandas as pd

import os
from collections import Counter

import matplotlib.pyplot as plt
import seaborn as sns
import statannot

from sklearn.utils import resample
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer

from lifelines import CoxPHFitter
from lifelines import KaplanMeierFitter

from lifelines.utils import concordance_index
from xgboost import XGBRegressor,XGBClassifier
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet

from keras.models import Sequential
from keras.layers import Dense, Dropout

from scipy.stats import mannwhitneyu,wilcoxon,shapiro, levene, ttest_ind,f_oneway,kruskal


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, normalize, StandardScaler
from sklearn.metrics import roc_curve, auc, accuracy_score, make_scorer,precision_recall_curve
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.svm import SVC
from abc import ABC, abstractmethod

import shap

import warnings
warnings.filterwarnings("ignore")


In [37]:
sample_info=pd.read_excel('../data/sample/personal_info.xlsx',index_col=0)
sample_df=pd.read_csv('../data/Using/sample.csv',index_col=0)
sample_info.index= pd.Series(sample_info.index).apply(
    lambda x: str(int(x)) if pd.notna(x) else np.nan
)
sample_info['hospital_id'] = sample_info['hospital_id'].apply(
    lambda x: int(x) if pd.notna(x) else np.nan
)

In [38]:
print(sample_info.shape)

(106, 5)


In [39]:
batch2_sample_info.index.tolist()

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan]

In [40]:
batch1_sample_info=sample_info[sample_info['hospital_id'].isna()]
batch2_sample_info=sample_info[sample_info.index.isna() & (~sample_info['hospital_id'].isna())]

In [41]:
print(batch2_sample_info.shape)
print(batch1_sample_info.shape)

(49, 5)
(57, 5)


In [42]:
batch2_sample_info

,hospital_id,gender,age,smoke,病理类型
Sample Name,,,,,
NaN,375613.0,男,57,是,单纯型
NaN,297162.0,男,65,是,单纯型
NaN,406143.0,男,59,是,单纯型
NaN,328827.0,男,61,是,单纯型
NaN,343494.0,男,56,是,复合型
NaN,551468.0,男,63,是,复合型
NaN,234806.0,男,52,是,单纯型
NaN,248795.0,女,55,否,复合型
NaN,261639.0,男,66,是,复合型


In [43]:
sample_to_id=sample_df[~sample_df.index.isna()]['hospital_id'].to_dict()
id_to_sample=sample_df.reset_index().set_index('hospital_id')[~sample_df.index.isna()]['Sample Name'].to_dict()

In [44]:
batch2_sample_info.index=batch2_sample_info['hospital_id'].map(id_to_sample)
batch1_sample_info['hospital_id']=batch1_sample_info.reset_index()['Sample Name'].map(sample_to_id).tolist()
batch2_sample_info.index.name='Sample Name'

In [45]:
batch1_sample_info

,hospital_id,gender,age,smoke,病理类型
Sample Name,,,,,
1272,297034,男,57,是,单纯型
1277,297125,男,68,是,单纯型
1334,299585,男,68,是,单纯型
1452,304077,男,68,是,单纯型
1520,305980,男,60,否,单纯型
1775,316661,男,60,是,单纯型
1870,320582,女,68,否,单纯型
2026,324455,男,67,是,单纯型
2039,322148,男,37,否,单纯型


In [46]:
batch2_sample_info

,hospital_id,gender,age,smoke,病理类型
Sample Name,,,,,
130190-T,375613.0,男,57,是,单纯型
354228-T,297162.0,男,65,是,单纯型
140279-T,406143.0,男,59,是,单纯型
120136-T,328827.0,男,61,是,单纯型
2536-T,343494.0,男,56,是,复合型
180041-T,551468.0,男,63,是,复合型
154264-T,234806.0,男,52,是,单纯型
173161-T,248795.0,女,55,否,复合型
196633-T,261639.0,男,66,是,复合型


In [57]:
whole_sample_info=pd.concat([batch1_sample_info,batch2_sample_info])
whole_sample_info.index.name='Sample Name'
whole_sample_info

,hospital_id,gender,age,smoke,病理类型
Sample Name,,,,,
1272,297034.0,男,57,是,单纯型
1277,297125.0,男,68,是,单纯型
1334,299585.0,男,68,是,单纯型
1452,304077.0,男,68,是,单纯型
1520,305980.0,男,60,否,单纯型
...,...,...,...,...,...
210012-T,2033135.0,男,59,否,单纯型
200095-T,643779.0,男,60,是,复合型
200097-T,646890.0,男,82,否,单纯型


In [58]:
# whole_sample_info.to_csv('../data/sample/personal_info.csv')